In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('Complaints.csv')

In [4]:
precincts = pd.DataFrame(columns = ['precinct'])

In [5]:
precincts['precinct'] = np.sort(df['precinct'].unique())

In [6]:
precincts = precincts.join(df.groupby(['precinct']).size().to_frame(name = 'Number of Complaints'),on = 'precinct')

In [7]:
precincts.where(precincts['precinct'] == 22).dropna()

,precinct,Number of Complaints
12,22.0,12.0


In [8]:
keys = precincts['precinct'].unique()

In [9]:
values = [190,141,164,142,178,162,190,307,141,294,207,
         149,120,176,156,210,134,167,142,205,191,201,
         284,186,221,293,358,174,306,280,252,193,133,
         282,202,167,154,155,138,333,138,161,303,216,
         179,275,399,139,235,198,247,197,209,221,155,
         185,142,145,206,209,254,182,310,225,197,148,
         292,211,153,144,236,261,234,311,196,195,144]

In [ ]:
UniformedDict = dict(zip(keys,values))
def descentToString(precinct):
    # Convert 1 letter abbreviations to their meaning using a Dict
    return UniformedDict[precinct]